In [ ]:
import networkx as nx
import json
import numpy as np
from scipy.spatial.distance import cosine
import statistics

In [1]:
import pickle



# marketing
#ent_fv_full = pickle.load(open("/home/arjun/NER_experiments/code/entity_graph_builder/graph_dumps/marketing_entity_feats_marketing_model_epc3.pkl","rb"))
#ent_graph = pickle.load(open("/home/arjun/NER_experiments/code/entity_graph_builder/graph_dumps/entity_kp_graph_marketing.pkl","rb"))

# S.E
ent_fv_full = pickle.load(open("/home/arjun/NER_experiments/code/entity_graph_builder/graph_dumps/se_entity_feats_se_model_v2epc3.pkl", "rb"))
#ent_graph = pickle.load(open("/home/ether/hdd/Venkat/knowledge_graphs/entity_graph_builder/graph_dumps/pruned_entity_kp_graph.pkl","rb"))
ent_graph = pickle.load(open("/home/arjun/NER_experiments/code/entity_graph_builder/graph_dumps/entity_kp_graph_directed_sev2_with_synrel.pkl", "rb"))
# Ether Graph
# ent_graph = pickle.load(open("/home/ether/hdd/Venkat/knowledge_graphs/entity_graph_builder/graph_dumps/se_ether_graph_slack_extended.pkl","rb"))
# ent_fv_full = pickle.load(open("/home/ether/hdd/Venkat/knowledge_graphs/entity_graph_builder/graph_dumps/ether_engg_entity_feats_+slack_ether_model_2+1_epc3.pkl","rb"))



common_entities = ent_fv_full.keys() & ent_graph.nodes()
ent_fv = {}
for ent in common_entities:
    ent_fv[ent] = ent_fv_full[ent]

In [3]:

G = nx.Graph()

G.add_nodes_from(list(ent_fv.keys()))

In [ ]:
for nodea in G.nodes():
    for nodeb in G.nodes():
        G.add_edge(nodea, nodeb, weight= 1-cosine(ent_fv[nodea], ent_fv[nodeb]))

In [ ]:
def construct_graph_ns_max(self, meeting_graph):
        
        max_connection = {}
        max_score = {}
        outlier_score = {}
        for node in meeting_graph.nodes():
            closest_connection_n = sorted(dict(meeting_graph[node]).items(), key=lambda kv:kv[1]["weight"], reverse=True)
            max_score_current = []
            
            max_connection[node] = closest_connection_n
            weights_n = list(map(lambda kv: (kv[1]["weight"]).tolist(), closest_connection_n))
            q3 = np.percentile(weights_n, 75)
            iqr = np.subtract(*np.percentile(weights_n, [75, 25]))
            outlier_score[node] = {}
            outlier_score[node]["outlier"] = q3 + 1 * iqr
            outlier_score[node]["iqr"] = iqr
            outlier_score[node]["q3"] = q3
            outlier_score[node]["weights_n"] = closest_connection_n
            outlier_score[node]["avg+pstd"] = statistics.mean(weights_n)+statistics.pstdev(weights_n)


        graph_data = deepcopy(meeting_graph.edges.data())
        for nodea, nodeb, weight in graph_data:
            if weight["weight"] > outlier_score[nodea]["outlier"] :
                pass
            else:
                meeting_graph.remove_edge(nodea, nodeb)

        return meeting_graph, yetto_prune


In [ ]:
meeting_graph = construct_graph_ns_max(G)